In [1]:
import json
import os

In [3]:
train_path = "../../ReadingBank/train/"

os.listdir("../../ReadingBank/train/")

['dataset-train-s2s-text-m3.json',
 'dataset-train-s2s-layout-m7.json',
 'dataset-train-s2s-text-m2.json',
 'dataset-train-s2s-text-m6.json',
 'dataset-train-s2s-text-m4.json',
 'dataset-train-s2s-text-m7.json',
 'dataset-train-s2s-layout-m4.json',
 'dataset-train-s2s-layout-m1.json',
 'dataset-train-s2s-layout-m3.json',
 'dataset-train-s2s-layout-m6.json',
 'dataset-train-s2s-text-m1.json',
 'dataset-train-s2s-layout-m2.json']

In [4]:
example_text_file = 'dataset-train-s2s-text-m3.json'
example_layout_file = 'dataset-train-s2s-layout-m3.json'

example_text_path = os.path.join(train_path, example_text_file)
exampel_layout_path = os.path.join(train_path, example_layout_file)

In [13]:
with open(example_text_path, mode="r") as f:
    text_data_string = f.read()

str(text_data_string)[:2239]

'{"src": "Expected Observed Area Action Pass/ Fail Comments Outcome Outcome Log into iPassport (as - - - - - - the alternate user) Menu Click the Profile The User Details tab option in the header should load Click the View My The staff profile Staff Profile link should load for the logged in user on the General tab Check the information You should see “You message in the have control over middle of the screen who can view certain personal information stored about you in iPassport…” Check under the There should be a information message field called Restrict visibility of these field fields to Click the pencil edit The field should icon for the Restrict expand visibility of these fields to field Check the items in There should be four the drop down items", "tgt": "Area Action Expected Outcome Observed Outcome Pass/ Fail Comments Log into iPassport (as the alternate user) - - - - - - Menu Click the Profile option in the header The User Details tab should load Click the View My Staff Profi

In [17]:
for dp in text_data_string.split("\n"):
    text_data = json.loads(dp)

    for key, value in text_data.items():
        print(key, ":", sep="")
        print(str(value))
        print()

    break

src:
Expected Observed Area Action Pass/ Fail Comments Outcome Outcome Log into iPassport (as - - - - - - the alternate user) Menu Click the Profile The User Details tab option in the header should load Click the View My The staff profile Staff Profile link should load for the logged in user on the General tab Check the information You should see “You message in the have control over middle of the screen who can view certain personal information stored about you in iPassport…” Check under the There should be a information message field called Restrict visibility of these field fields to Click the pencil edit The field should icon for the Restrict expand visibility of these fields to field Check the items in There should be four the drop down items

tgt:
Area Action Expected Outcome Observed Outcome Pass/ Fail Comments Log into iPassport (as the alternate user) - - - - - - Menu Click the Profile option in the header The User Details tab should load Click the View My Staff Profile link T

In [19]:
with open(exampel_layout_path, mode="r") as f:
    layout_data_string = f.read()

str(layout_data_string)[:2239]

for dp in layout_data_string.split("\n"):
    layout_data = json.loads(dp)

    for key, value in layout_data.items():
        print(key, ":", sep="")
        print(str(value))
        print()

    break

src:
[[372, 278, 435, 305], [511, 278, 577, 305], [107, 289, 139, 317], [241, 289, 285, 317], [627, 289, 664, 317], [668, 289, 692, 317], [791, 289, 865, 317], [373, 301, 434, 328], [513, 301, 575, 328], [200, 325, 221, 350], [225, 325, 246, 350], [250, 325, 304, 350], [308, 325, 326, 350], [340, 325, 344, 350], [348, 325, 352, 350], [356, 325, 360, 350], [364, 325, 368, 350], [372, 325, 376, 350], [380, 325, 384, 350], [200, 346, 218, 372], [222, 346, 272, 372], [276, 346, 306, 372], [59, 368, 92, 394], [200, 368, 228, 394], [232, 368, 250, 394], [254, 368, 294, 394], [340, 368, 362, 394], [366, 368, 394, 394], [397, 368, 437, 394], [441, 368, 459, 394], [200, 389, 235, 415], [239, 389, 249, 415], [253, 389, 271, 415], [275, 389, 315, 415], [340, 389, 378, 415], [382, 389, 407, 415], [200, 412, 228, 437], [232, 412, 250, 437], [254, 412, 283, 437], [287, 412, 305, 437], [340, 412, 362, 437], [366, 412, 391, 437], [395, 412, 430, 437], [200, 433, 229, 458], [232, 433, 273, 458], [277, 

In [22]:
from docx import Document

docx_folder_path = "../../docx_cvs"

document = Document()
document.add_paragraph("It was a dark and stormy night.")
document.save(os.path.join(docx_folder_path, "dark-and-stormy.docx"))

document = Document(os.path.join(docx_folder_path, "dark-and-stormy.docx"))
document.paragraphs[0].text

'It was a dark and stormy night.'

In [25]:
docx_file = "new_resume_001.docx"

docx_path = os.path.join(docx_folder_path, docx_file)


# https://python-docx.readthedocs.io/en/latest/

document = Document(docx_path)
document

# Make CUSTOM DATA

In [ ]:
# The threads (en de linked): https://github.com/microsoft/unilm/issues/464
# deze functie: https://github.com/microsoft/unilm/blob/master/layoutreader/s2s_ft/utils.py#L402



import logging
from typing import Tuple, List
import os
import numpy as np
import torch
from transformers import AutoTokenizer

import s2s_ft.s2s_loader as seq2seq_loader
from s2s_ft.modeling_decoding import LayoutlmForSeq2SeqDecoder, BertConfig
from s2s_ft.s2s_loader import Preprocess4Seq2seqDecoder


class TextOrderer:
    def __init__(self, model_path: str):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # pylint: disable=no-member
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlmv2-base-uncased")
        config_file = os.path.join(model_path, "config.json")

        self.config = BertConfig.from_json_file(config_file, layoutlm_only_layout_flag=True)
        self.model = LayoutlmForSeq2SeqDecoder.from_pretrained(model_path, config=self.config).to(self.device)
        self.max_len = 511
        self.preprocessor = Preprocess4Seq2seqDecoder(
            list(self.tokenizer.vocab.keys()),
            self.tokenizer.convert_tokens_to_ids,
            1024,
            max_tgt_length=self.max_len,
            layout_flag=True
        )

    def __call__(self, *args, **kwargs):
        return self.reconstruct(*args, **kwargs)

    def forward(self, words, boxes) -> List[int]:
        """
        :param words: Word list [sorted in top-down / left-right fashion for best performance)
        :param boxes: Normalized bounding box list (layoutlm format)
        :return: Re-ordered index list
        """
        assert len(words) == len(boxes)

        instance = [[x[0], *x[1]] for x in list(zip(words, boxes))], len(boxes)
        instances = [self.preprocessor(instance)]
        with torch.no_grad():
            batch = seq2seq_loader.batch_list_to_batch_tensors(
                instances)
            batch = [
                t.to(self.device) if t is not None else None for t in batch]
            input_ids, token_type_ids, position_ids, input_mask, mask_qkv, task_idx = batch

            traces = self.model(input_ids, token_type_ids,
                                position_ids, input_mask, task_idx=task_idx, mask_qkv=mask_qkv)
            output_ids = traces.squeeze().tolist()
            output_ids = list(np.array(output_ids) - 1)
            return output_ids


    def reconstruct(self, words: List[str], boxes: List[List[int]]) -> Tuple[List[str], List[List[int]]]:

        assert len(words) == len(boxes)

        if len(words) > self.max_len:
            logging.warning(
                f"Page contains {len(words)} words. Exceeds the {self.max_len} limit and will not be reordered.")
            return words, boxes

        try:
            idx = self.forward(words, boxes)
            processed_idx = list(dict.fromkeys(idx))
            if len(processed_idx) != len(words):
                processed_idx = [idx for idx in processed_idx if idx < len(words)]
                unused_idx = sorted(list(set(range(len(words))) - set(processed_idx[:len(words)])))
                logging.info(
                    f"There is {len(words)} words but only {len(processed_idx)} indexes. "
                    f"Unmatched indexes: {unused_idx}")
                processed_idx.extend(unused_idx)
                logging.info(f"There is now {len(words)} wordsand {len(processed_idx)} indexes.")
                assert len(processed_idx) == len(words)

            words = list(np.array(words)[processed_idx])
            boxes = [elem.tolist() for elem in np.array(boxes)[processed_idx]]
            return words, boxes

        except Exception as exception: # pylint: disable=broad-except
            logging.warning(exception)
            return words, boxes
        



    # convert pdf naar words: List[str], boxes: List[List[int]]
        
    def extract_text_with_bounding_boxes(pdf_path):
        words = []
        bounding_boxes = []
        
        for page_layout in extract_pages(pdf_path):
            for element in page_layout:
                if isinstance(element, LTTextContainer):
                    for text_line in element:
                        if isinstance(text_line, LTTextLine):
                            word = text_line.get_text().strip()  # Get the text of the line
                            bounding_box = text_line.bbox  # Get the bounding box of the line
                            bounding_box = [round(coord) for coord in bounding_box]  # Round the coordinates
                            words.append(word)
                            bounding_boxes.append(bounding_box)
        
        return words, bounding_boxes

# dan kan je reconstruct aanroepen
# dan dataset
# en dan met layoutreader